# <font color='black'> Методы анализа неоднородных данных и паттерн-анализ. Практическое занятие 4 </font>
## <font color='black'> Анализ панельных данных. Модели с фиксированными эффектами (FE-models) и модели со случайными эффектами (Часть 2) </font>

На этом занятии мы продолжим работать с панельными данными, источником которых является исследование Ehrlich I., Lui F. (1999) Bureaucratic Corruption and Endogenous Economic Growth. The Journal of Political Economy, 107 (6), pp. 270 – 293.

Краткое описание данных:
* country -  Страна
* countrygroup - Страны в датасете разделены на три группы. Первая группа (countrygroup = 1) включает Австралию, Канаду, Японию, Новую Зеландию и США. Вторая группа (countrygroup = 2) включает страны Восточной Европы (Болгария, Хорватия, Чешская Республика, Эстония, Венгрия, Латвия, Польша, Словения, Словакия). В третью группу в основном входят страны Западной Европы
* year - Год. Данные охватывают временной промежуток с 2006 по 2015 год
* pol_stab - Политическая стабильность и отсутствие насилия/терроризма. Отражает склонность к политической нестабильности и/или политически мотивированному насилию, включая терроризм. Значения показателя варьируются от −2.5 до 2.5, причем более высокие значения означают более высокий уровень политической стабильности. Источником данных являются Индикаторы качества государственного управления (WGI). В рамках анализа будем использовать этот показатель в качестве зависимой переменной
* con_cor - Показатель контроля коррупции. Отражает восприятие степени использования государственной власти для личной выгоды, включая как мелкие, так и крупные формы коррупции. Показатель изменяется от −2.5 до 2.5, причем более высокие зна-чения соответствуют более низкому уровню коррупции (т.е. более высокому контролю). Источником данных является WGI. В рамках анализа будем использовать эту переменную в качестве ключевого предиктора
* herfgov_DPI - Индекс концентрации Херфиндаля. В контексте данного исследования он используется для измерения уровня конкуренции между политическими партиями. Шкала варьируется от 0 до 1, где 1 означает отсутствие конкуренции. В рамках анализа будем использовать как контрольную переменную
* govt_consump_WDI - Государственные расходы на конечное потребление (в процентах от ВВП). В рамках анализа будем использовать как контрольную переменную


Подгрузим необходимые библиотеки и откроем массив "MApolit_lab3.dta".

In [ ]:
import pandas as pd
import statsmodels.formula.api as statf
import numpy as np
import numpy.linalg as la
!pip install linearmodels
from linearmodels import PanelOLS
from linearmodels import RandomEffects
from scipy import stats

In [ ]:
lab3 = pd.read_stata("MApolit_lab3.dta")
lab3 = lab3.dropna()
lab3.head()

Вспомним, что можем представить модель с фиксированными эффектами как модель с набором дамми-переменных. В данном случае нас будет интересовать фиксированные эффекты на пространственные единицы, а именно страны.

In [ ]:
model1 = statf.ols(formula = 'pol_stab ~ con_cor + govt_consump_WDI + herfgov_DPI + C(country)', data = lab3).fit()
print(model1.summary())

Ту же модель можно представить в более экономном варианте, используя внутригрупповое преобразование (центрирование как зависимой переменной, так и предикторов по подгруппам).

In [ ]:
lab3_index = lab3.set_index(['country', 'year'])
model1_1 = PanelOLS.from_formula('pol_stab ~ con_cor + govt_consump_WDI + herfgov_DPI + EntityEffects', data = lab3_index).fit()
print(model1_1)

Проверим, хорошо ли предсказывает модель значения политической стабильности. При этом так как мы работаем с панельным массивом, нас будут интересовать предсказание с разбиением на подгруппы - страны. Оценим корреляцию зависимой переменной (наблюдаемого отклика) и предсказанного значения отклика по подгруппам. Если корреляция низкая (по модулю), то это говорит о том, что мы плохо объясняем моделью то, что происходит в данной группе (стране). Переоценим модель на подвыборке тех стран, для которых корреляция по модулю более либо равна 0.3 и посмотрим, устойчивы ли наши результаты.  

In [ ]:
lab3['ypredicted'] = model1.predict()

groups_data = pd.DataFrame(lab3.groupby('country')['ypredicted'].corr(lab3['pol_stab']))
groups_data['group'] = groups_data['ypredicted'].apply(lambda x: 1 if abs(x) >= 0.1 else 0)
data_merged = pd.merge(lab3, groups_data, on = 'country')
model_high = statf.ols(formula = 'pol_stab ~ con_cor + govt_consump_WDI + herfgov_DPI + C(country)', data = data_merged[data_merged['group']==1]).fit()
print(model_high.summary())

Сравним полученные оценки с результатами оценивания модели со случайными эффектами.
* Вспомните допущения данной модели и оцените критически
* Какой метод оценивания используется в данном случае?

In [ ]:
model2 = RandomEffects.from_formula('pol_stab ~ 1 + con_cor + govt_consump_WDI + herfgov_DPI', data = lab3_index).fit()
print(model2)

Для начала приведите содержательные основания для выбора FE- или RE-модели: какой модели Вы отдадите предпочтение в данном случае?

Но мы также обсуждали с Вами тест Хаусмана и его ограничения для сравнения FE- и RE-модели.
Зададим в явном виде статистику для проверки гипотезы  и ее распределение: $$S = (\hat{b}_{fe} - \hat{b}_{re})^T(Cov({\hat{b}_{fe}}) - Cov(\hat{b}_{re}))^{-1}(\hat{b}_{fe} - \hat{b}_{re}) \sim \chi^2_{k}$$, где k - количество предикторов. Проинтерпретируйте результаты тестирования. Как Вы думаете, с чем связаны полученные результаты?

In [ ]:
X_varying = ['con_cor', 'govt_consump_WDI', 'herfgov_DPI']

fe_coef = model1_1.params
re_coef = model2.params.loc[X_varying]

fe_cov = model1_1.cov
re_cov = model2.cov[X_varying].loc[X_varying]

hausman = (fe_coef - re_coef).dot(la.inv(fe_cov - re_cov).dot(fe_coef - re_coef))
df = fe_coef.size
pvalue = stats.chi2.sf(hausman, df)
print('Chi-squared:', hausman, '\n' 'P-value:', pvalue, '\n' 'Degrees of freedom:', df)

В следующей модели давайте учтем фиксированные эффекты на временные периоды.
* Проинтерпретируйте хотя бы один коэффициент при временном периоде
* Нужны ли в данном случае FE на временные периоды?
* Что теперь отражают оценки коэффициентов при предикторах? Какую вариацию?

In [ ]:
model3 = statf.ols(formula = 'pol_stab ~ con_cor + govt_consump_WDI + herfgov_DPI + C(year)', data = lab3).fit()
print(model3.summary())

Та же модель, но с внутригрупповым преобразованием:

In [ ]:
model3_1 = PanelOLS.from_formula('pol_stab ~ con_cor + govt_consump_WDI + herfgov_DPI + TimeEffects', lab3_index).fit()
print(model3_1)

Оценим twoways модель: учтем и FE на год, и FE на страны. В чем заключаются ограничения этой модели?

In [ ]:
model4 = statf.ols('pol_stab ~ con_cor + govt_consump_WDI + herfgov_DPI + C(country) + C(year)', lab3).fit()
print(model4.summary())

Сравним model1 и model4. Эти модели являются вложенными, так как можно получить спецификацию второй модели из первой посредством только добавления параметров (коэффициентов при дамми на годы). Какую модель предпочтительно выбрать?

In [ ]:
F = ((model4.rsquared - model1.rsquared)/(model4.params.size - model1.params.size))/((1-model4.rsquared)/(model4.nobs - model4.params.size))
F
pvalue = stats.f.sf(F, model4.params.size - model1.params.size, model4.nobs - model4.params.size)
print('F-statistic:', F, '\n' 'P-value:', pvalue, '\n' 'df1:', model4.params.size - model1.params.size, '\n' 'df2:', model4.nobs - model4.params.size)

Проверим, одинаковая ли взаимосвязь коррупции и политической стабильности в разных группах стран. Оценим varying-slope model. Проинтерпретируйте полученные результаты.  

Примечание: до этого мы рассматривали страну в качестве пространственной единицы. Однако в таком случае есть опасность получить чрезмерно фрагментарные результаты, что будет сопровождаться безумным количеством параметров и сложностями в интерпретации. Вместо этого оценим модель с разными наклонами на группы стран.

In [ ]:
model5 = statf.ols('pol_stab ~ con_cor*C(countrygroup) + govt_consump_WDI + herfgov_DPI', lab3).fit()
print(model5.summary())